<a href="https://colab.research.google.com/github/kmouleejunhyuk/bigdata_analysist_practical/blob/main/statsmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#prepare data
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
#@title  { display-mode: "both" }
# 단계 1: 폰트 설치
import matplotlib.font_manager as fm

!apt-get -qq -y install fonts-nanum > /dev/null
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
fm._rebuild() 

In [ ]:
#@title
# 단계 2: 런타임 재시작
import os
os.kill(os.getpid(), 9)

In [1]:
#@title
# 단계 3: 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 마이너스 표시 문제
mpl.rcParams['axes.unicode_minus'] = False
	
# 한글 폰트 설정
path = '/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf'
font_name = fm.FontProperties(fname=path, size=18).get_name()
plt.rc('font', family=font_name)
fm._rebuild() 

In [2]:
PATH ='/gdrive/MyDrive/big_2021/data/ADP'
SEED = 49

In [25]:
import pandas as pd
import numpy as np
import collections
import sklearn
import statsmodels.api as stats

In [4]:
rawdata = pd.read_csv(PATH + '/PART 05 실습용 데이터/credit_final.csv', encoding = 'cp949')

In [65]:
rawdata.head()

,credit.rating,account.balance,credit.duration.months,previous.credit.payment.status,credit.purpose,credit.amount,savings,employment.duration,installment.rate,marital.status,guarantor,residence.duration,current.assets,age,other.credits,apartment.type,bank.credits,occupation,dependents,telephone,foreign.worker
0,1,1,18,3,2,1049,1,1,4,1,1,4,2,21,2,1,1,3,1,1,1
1,1,1,9,3,4,2799,1,2,2,3,1,2,1,36,2,1,2,3,2,1,1
2,1,2,12,2,4,841,2,3,2,1,1,4,1,23,2,1,1,2,1,1,1
3,1,1,12,3,4,2122,1,2,3,3,1,2,1,39,2,1,2,2,2,1,2
4,1,1,12,3,4,2171,1,2,4,3,1,4,2,38,1,2,2,2,1,1,2


회귀분석

In [64]:
col = [(s, s.replace('.', '_')) for s in rawdata.columns]

In [62]:
r_modified = rawdata.rename(dict(col), axis = 1)

In [66]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(r_modified, random_state = SEED, test_size = 0.2, shuffle = True)

In [67]:
train.shape, test.shape

((800, 21), (200, 21))

In [69]:
x_var = " + ".join(train.columns[1:])

In [71]:
reg = stats.OLS.from_formula("credit_rating ~ " + x_var, data = train)
print(reg.fit().summary())

                            OLS Regression Results                            
Dep. Variable:          credit_rating   R-squared:                       0.269
Model:                            OLS   Adj. R-squared:                  0.250
Method:                 Least Squares   F-statistic:                     14.34
Date:                Sun, 09 May 2021   Prob (F-statistic):           4.81e-41
Time:                        09:10:01   Log-Likelihood:                -387.41
No. Observations:                 800   AIC:                             816.8
Df Residuals:                     779   BIC:                             915.2
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

로지스틱회귀분석

In [111]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(rawdata, random_state = SEED, test_size = 0.2, shuffle = True)

In [112]:
train.shape, test.shape

((800, 21), (200, 21))

In [113]:
y_train = train['credit.rating']
x_train = train.loc[:, train.columns != 'credit.rating']

In [114]:
log_reg = stats.Logit(y_train, x_train).fit()

Optimization terminated successfully.
         Current function value: 0.465654
         Iterations 6


In [129]:
print(log_reg.summary2())

                                Results: Logit
Model:                   Logit                Pseudo R-squared:     0.240     
Dependent Variable:      credit.rating        AIC:                  785.0460  
Date:                    2021-05-09 10:18     BIC:                  878.7383  
No. Observations:        800                  Log-Likelihood:       -372.52   
Df Model:                19                   LL-Null:              -490.37   
Df Residuals:            780                  LLR p-value:          2.3979e-39
Converged:               1.0000               Scale:                1.0000    
No. Iterations:          6.0000                                               
------------------------------------------------------------------------------
                                Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
------------------------------------------------------------------------------
account.balance                 0.8516   0.1115  7.6398 0.0000  0.6331  1.0701
credi

In [116]:
y_test = test['credit.rating']
x_test = test.loc[:, test.columns != 'credit.rating']

In [125]:
from sklearn.metrics import roc_auc_score
y_hat = log_reg.predict(x_test)
p = [1 if x > 0.7 else 0 for x in log_reg_sel.predict(x_test[selected]).values]
print(roc_auc_score(y_test, p))

0.6934191355026712


로지스틱_변수 제외

In [126]:
selected = [x for x in train.columns if x not in ['credit.rating', 'credit_amount', 'employment.duration', 'guarantor', 'residence.duration', 'age', 'other.credits', 'apartment.type', 'bank_credits', 'occupation', 'dependents', 'telephone', 'foreign.worker']]
y_train = train['credit.rating']
x_train = train[selected]

In [132]:
log_reg_sel = stats.Logit(y_train, x_train).fit()
print(log_reg_sel.summary())
y_hat = log_reg_sel.predict(x_test[selected])
p = [1 if x > 0.7 else 0 for x in log_reg_sel.predict(x_test[selected]).values]
print(roc_auc_score(y_test, p))

Optimization terminated successfully.
         Current function value: 0.477132
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:          credit.rating   No. Observations:                  800
Model:                          Logit   Df Residuals:                      790
Method:                           MLE   Df Model:                            9
Date:                Sun, 09 May 2021   Pseudo R-squ.:                  0.2216
Time:                        10:19:25   Log-Likelihood:                -381.71
converged:                       True   LL-Null:                       -490.37
Covariance Type:            nonrobust   LLR p-value:                 7.598e-42
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
account.balance                    0.8751      0.108      8.121     